In [40]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split as holdout
from sklearn.linear_model import LinearRegression
from sklearn import metrics

import numpy as np

import joblib

#  Import and read the charity_data.csv.
import pandas as pd 
complete_data_df= pd.read_csv("resources/avg_by_region.csv")
complete_data_df.head()


,id,age,average_income,sex,bmi,children,smoker,region,charges,average_age
0,0,18,10753.5,male,33.77,1,no,southeast,1725.5523,78.43
1,1,18,10753.5,male,34.10,0,no,southeast,1137.0110,78.43
2,5,18,10753.5,male,31.68,2,yes,southeast,34303.1672,78.43
3,9,18,10753.5,female,36.85,0,yes,southeast,36149.4835,78.43
4,11,18,10753.5,female,38.28,0,no,southeast,1631.8212,78.43


In [7]:
complete_data_df.nunique()

id                1338
age                 47
average_income      47
sex                  2
bmi                548
children             6
smoker               2
region               4
charges           1337
average_age          4
dtype: int64

In [8]:
complete_data_cat = complete_data_df.dtypes[complete_data_df.dtypes == "object"].index.tolist()
complete_data_df

,id,age,average_income,sex,bmi,children,smoker,region,charges,average_age
0,0,18,10753.50,male,33.770,1,no,southeast,1725.55230,78.43
1,1,18,10753.50,male,34.100,0,no,southeast,1137.01100,78.43
2,5,18,10753.50,male,31.680,2,yes,southeast,34303.16720,78.43
3,9,18,10753.50,female,36.850,0,yes,southeast,36149.48350,78.43
4,11,18,10753.50,female,38.280,0,no,southeast,1631.82120,78.43
...,...,...,...,...,...,...,...,...,...,...
1333,1316,64,87985.91,male,24.700,1,no,northwest,30166.61817,79.88
1334,1321,64,87985.91,female,30.115,3,no,northwest,16455.70785,79.88
1335,1323,64,87985.91,female,32.965,0,no,northwest,14692.66935,79.88
1336,1330,64,87985.91,male,37.905,0,no,northwest,14210.53595,79.88


In [9]:
complete_data_df.head(5)

,id,age,average_income,sex,bmi,children,smoker,region,charges,average_age
0,0,18,10753.5,male,33.77,1,no,southeast,1725.5523,78.43
1,1,18,10753.5,male,34.10,0,no,southeast,1137.0110,78.43
2,5,18,10753.5,male,31.68,2,yes,southeast,34303.1672,78.43
3,9,18,10753.5,female,36.85,0,yes,southeast,36149.4835,78.43
4,11,18,10753.5,female,38.28,0,no,southeast,1631.8212,78.43


In [17]:
dummies = pd.get_dummies(complete_data_df[['sex', 'smoker', 'region']], drop_first=True)
text_features = pd.concat([complete_data_df.drop(['sex', 'smoker', 'region'],axis=1), dummies],axis=1)
text_features.head()

,id,age,average_income,bmi,children,charges,average_age,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,0,18,10753.5,33.77,1,1725.5523,78.43,1,0,0,1,0
1,1,18,10753.5,34.10,0,1137.0110,78.43,1,0,0,1,0
2,5,18,10753.5,31.68,2,34303.1672,78.43,1,1,0,1,0
3,9,18,10753.5,36.85,0,36149.4835,78.43,0,1,0,1,0
4,11,18,10753.5,38.28,0,1631.8212,78.43,0,0,0,1,0


In [37]:
x = text_features.drop(['charges', 'sex_male', 'region_northwest', 'region_southeast', 'region_southwest'], axis = 1)
y = text_features.charges

# changing the number of degrees will reduce the predicted value. 2 is the optimal number of degrees 
pol = PolynomialFeatures (degree = 2)
x_pol = pol.fit_transform(x)

x_train, x_test, y_train, y_test = holdout(x_pol, y, test_size=0.2, random_state=0)
Pol_reg = LinearRegression()
Pol_reg.fit(x_train, y_train)
y_train_pred = Pol_reg.predict(x_train)
y_test_pred = Pol_reg.predict(x_test)
print(Pol_reg.intercept_)
print(Pol_reg.coef_)
print(Pol_reg.score(x_test, y_test))

-85945.74490220737
[-2.50051568e-03  4.83902722e+02 -1.26745744e+04  4.71695139e-01
 -1.86283473e+03 -9.44562336e+03  4.73079951e+03 -6.70833151e+03
  1.55473478e-01 -1.07581110e+01  8.08422813e-04  1.77268035e+00
  4.79522569e+00 -4.64753429e+00 -5.04787445e-01  1.73841133e+02
 -1.62300158e-02 -4.59335938e+01 -9.21397135e+01  1.11315358e+02
 -1.34368063e+02 -1.53870553e-06 -1.97597422e-03 -2.83390498e-02
 -2.19381832e-03  7.99995373e-02 -6.88048356e+00 -1.20340442e+01
  3.87433802e+01  1.46395250e+03 -1.32394030e+02  1.69154902e+02
 -8.15122170e+02 -4.40908239e+01 -7.75134760e+01 -6.70833121e+03]
0.8115484975992052


In [22]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_test_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_test_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

Mean Absolute Error: 3234.4259685018415
Mean Squared Error: 27325772.327523492
Root Mean Squared Error: 5227.4058889207645


In [38]:
# pridicting the charges 
y_test_pred = Pol_reg.predict(x_test)

#compare the acutal value with our predicted value 
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred})
df

,Actual,Predicted
578,8627.54110,9135.719141
610,24869.83680,26478.129090
569,8334.58960,10118.372661
1034,17748.50620,24838.950090
198,5699.83750,7331.975674
...,...,...
1084,15817.98570,19011.517366
726,1964.78000,3757.631643
1132,4454.40265,6164.092615
725,38344.56600,40596.788013


In [44]:
y_test_pred.save_model("best_model.json")

AttributeError: 'numpy.ndarray' object has no attribute 'save_model'